In [8]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = "mightytranks@gmail.com"
my_password = "OsGiwuevzRf4925Lfty!"
my_app_key = "X9EGwt5zbpg8o5xk"

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()


<LoginResource>

In [9]:
# Grab all event type ids. This will return a list which we will iterate over to print out the id and the name of the sport
# event_types = trading.betting.list_event_types()

# sport_ids = pd.DataFrame({
#     'Sport': [event_type_object.event_type.name for event_type_object in event_types],
#     'ID': [event_type_object.event_type.id for event_type_object in event_types]
# }).set_index('Sport').sort_index()

# sport_ids

In [10]:
gh_racing_id = 4339
# Filter for just horse racing
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_type_codes=['FORECAST'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ")
    }
)
greyhound_racing_filter

{'eventTypeIds': [4339],
 'marketCountries': ['GB'],
 'marketTypeCodes': ['FORECAST'],
 'marketStartTime': {'to': '2022-06-16T19:59:54Z'}}

In [11]:
#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)

# Create a DataFrame with all the events by iterating over each event object
gb_gh_events_df = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in gb_gh_events],
    'Event ID': [event_object.event.id for event_object in gb_gh_events],
    'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
    'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
    'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
    'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
    'Market Count': [event_object.market_count for event_object in gb_gh_events]
})

gb_gh_events_df

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Perry Barr (F/C) 16th Jun,31529845,Perry Barr,GB,Europe/London,2022-06-16 17:26:00,10
1,Crayford (F/C) 16th Jun,31529796,Crayford,GB,Europe/London,2022-06-16 12:49:00,10
2,Swindon (F/C) 16th Jun,31529911,Swindon,GB,Europe/London,2022-06-16 14:54:00,10
3,Hove (F/C) 16th Jun,31529863,Hove,GB,Europe/London,2022-06-16 17:09:00,10
4,Romford (F/C) 16th Jun,31529807,Romford,GB,Europe/London,2022-06-16 12:57:00,10
5,Newcastle (F/C) 16th Jun,31529774,Newcastle,GB,Europe/London,2022-06-16 13:04:00,9
6,Monmore (F/C) 16th Jun,31529854,Monmore,GB,Europe/London,2022-06-16 17:17:00,9
7,Sunderland (F/C) 16th Jun,31529822,Sunderland,GB,Europe/London,2022-06-16 17:19:00,10


In [12]:
#TODO: extract a list of IDs for the forecast markets
fc_venue_ids = []
fc_venue_names = []
for eventObj in gb_gh_events:
    fc_venue_ids.append(eventObj.event.id)
    fc_venue_names.append(eventObj.event.name)

fc_venue_ids, fc_venue_names


(['31529845',
  '31529796',
  '31529911',
  '31529863',
  '31529807',
  '31529774',
  '31529854',
  '31529822'],
 ['Perry Barr (F/C) 16th Jun',
  'Crayford (F/C) 16th Jun',
  'Swindon (F/C) 16th Jun',
  'Hove (F/C) 16th Jun',
  'Romford (F/C) 16th Jun',
  'Newcastle (F/C) 16th Jun',
  'Monmore (F/C) 16th Jun',
  'Sunderland (F/C) 16th Jun'])

In [13]:
# ###
# # Get odds from NEDS
# ###
# from neds_scraper import get_neds_odds
# venueName = 'newcastle-bags'
# odds = get_neds_odds(venueName)
# print(odds)

In [15]:
# market_catalogue_filter = filters.market_filter(event_ids=[venueOfTheDay_id])
market_catalogue_filter = filters.market_filter(event_ids=fc_venue_ids)

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    max_results='10',
    sort='FIRST_TO_START'
)

market_types_venueOfTheDay_df = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
    'Venue': [market_cat_object.event for market_cat_object in market_catalogues]
    
})
market_types_venueOfTheDay_df
#### https://www.betfair.com.au/exchange/plus/greyhound-racing/market/<Market ID>
# market_types_venueOfTheDay_df.to_csv('out.zip', index=False)  

,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Forecast,1.200217622,None,0.0,None
1,Forecast,1.200217582,None,0.0,None
2,Forecast,1.200217664,None,0.0,None
3,Forecast,1.200217623,None,0.0,None
4,Forecast,1.200217583,None,0.0,None
5,Forecast,1.200217665,None,0.0,None
6,Forecast,1.200217624,None,0.0,None
7,Forecast,1.200217584,None,0.0,None
8,Forecast,1.200217666,None,0.0,None
9,Forecast,1.200218408,None,0.0,None


In [39]:
#TODO: extract a list of IDs for the forecast markets
fc_race_ids = []
fc_race_names = []
for raceObj in market_catalogues:
    fc_race_ids.append(raceObj.market_id)

fc_race_ids

['1.199936082',
 '1.199996462',
 '1.199995536',
 '1.199996220',
 '1.199996261',
 '1.199996463',
 '1.199996221',
 '1.199996262',
 '1.199996464',
 '1.199996222']

In [40]:
# pick the random race id
# random_pick = random.randint(1,len(fc_race_ids))
# myRaceID = fc_race_ids[0]
myRaceID = '1.199996220'

'1.199936082'

In [41]:
def process_runner_books(runner_books):
    '''
    This function processes the runner books and returns a DataFrame with the best back/lay prices + vol for each runner
    :param runner_books:
    :return:
    '''
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 1.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 1.01
                      for runner_book
                      in runner_books]
    
    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [42]:
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)

runners_df

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,41419585,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
1,41419586,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
2,41419587,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
3,41419588,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
4,41419589,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
5,41419592,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
6,41419593,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
7,41419594,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
8,41419595,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None
9,41419596,1.01,1.01,1000.0,1.01,None,None,LOSER,None,None


In [ ]:
fav_selection_id = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Selection ID']
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

limit_order_filter = filters.limit_order(
    size=5,
    price=3,
    persistence_type='LAPSE'
)

instructions_filter = filters.place_instruction(
    selection_id = str(fav_selection_id),
    order_type = "LIMIT",
    side="LAY",
    limit_order=limit_order_filter
)

instructions_filter

{'orderType': 'LIMIT',
 'selectionId': '39483384',
 'side': 'LAY',
 'limitOrder': {'price': 3, 'persistenceType': 'LAPSE', 'size': 5}}

In [ ]:
# order = trading.betting.place_orders(
#     market_id = myRaceID,
#     customer_strategy_ref='Naing_maker',
#     instructions=[instructions_filter]
# )

In [ ]:
# order.__dict__

In [ ]:
# trading.betting.list_current_orders(customer_strategy_refs=['Naing_maker']).__dict__

In [ ]:
# cancelled_order = trading.betting.cancel_orders(market_id=myRaceID)

In [ ]:
# # listClearedOrders
# cleared_orders = trading.betting.list_cleared_orders(bet_status="CANCELLED",
#                                                     market_ids=[myRaceID])
# # Create a DataFrame from the orders
# pd.DataFrame(cleared_orders._data['cancelledOrders'])
# cleared_orders